Los resultados necesarios son:   

**Problema Inverso**:  Tenemos que obtener la estimación de los parámetros, sin preocuparnos por nada más.  
1º Usando la PINN más básica, hacemos la estimación de varios valores de E, para varios valores de E inicial.     
2º Usando el truco de alpha, hacemos la estimación, lo mismo que antes.  
3º Usando la formulación mixta esa, hacemos lo mismo con lo anterior (con alpha si habia una diferencia sustancial).    
¿4º? Lambde?

**Problema Directo:** 
1º Sin tener que estimar ningún parámetro, sacamos las deformaciones.  
2º Sacando los parámetros sacamos las deformaciones y los valores del parámetros. 
3º Con la formulación mixta. Lo mismo.  

Importante solo imponer los tensores de tensiones conocidos alli donde hay fuerzas.

# 1. PROBLEMA INVERSO SENCILLO 